# Evaluate 
$ f(x, y) = 2x^2 - x^2y^2 + 3 $

$ g_0 = x ⋅ x $ 

$ g_1 = x ⋅ x $ 

$ g_2 = y ⋅ y $

$ g_3 = g_0 ⋅ 2 $

$ g_4 = g_1 ⋅ g_2 $

$ g_5 = g_2 - g_4 $

$ g_6 = g_4 + 3 $

---

$ c_0 = a_0 ⋅ b_0 $

$ c_1 = a_1 ⋅ b_1 $

$ c_2 = a_2 ⋅ b_2 $

$ c_3 = a_3 ⋅ 2 $

$ c_4 = a_4 ⋅ b_4 $

$ c_5 = a_5 - b_5 $

$ c_6 = a_6 + 3 $

---

$ x = a_0 = b_0 = a_1 = b_1 $

$ y = a_2 = b_2 $

$ g_0 = a_3 = c_0 $

$ g_1 = a_4 = c_1 $

$ g_2 = b_4 = c_2 $

$ g_3 = a_5 = c_3 $

$ g_4 = b_5 = c_4 $

$ g_5 = a_6 = c_5 $

$ g_6 = c_6 $

### Gate constrain equation

$ g_i = q_{Li}⋅a_i+q_{Ri}⋅b_i+q_{Mi}⋅(a_i⋅b_i)+q_{Ci}+q_{Oi}⋅c_i=0 $

$ g_0 = 0⋅a_0 + 0⋅b_0 + 1⋅(a_0⋅b_0) + 0 + (-1)⋅c_0 = 0 $

$ g_1 = 0⋅a_1 + 0⋅b_1 + 1⋅(a_1⋅b_1) + 0 + (-1)⋅c_1 = 0 $

$ g_2 = 0⋅a_2 + 0⋅b_2 + 1⋅(a_2⋅b_2) + 0 + (-1)⋅c_2 = 0 $

$ g_3 = 2⋅a_3 + 0⋅b_3 + 0⋅(a_3⋅b_3) + 0 + (-1)⋅c_3 = 0 $

$ g_4 = 0⋅a_4 + 0⋅b_4 + 1⋅(a_4⋅b_4) + 0 + (-1)⋅c_4 = 0 $

$ g_5 = 1⋅a_5 + (-1)⋅b_5 + 0⋅(a_5⋅b_5) + 0 + (-1)⋅c_5 = 0 $

$ g_6 = 1⋅a_6 + 0⋅b_6 + 0⋅(a_6⋅b_6) + 3 + (-1)⋅c_6 = 0 $

**selector vectors**

$ g_i = [q_{Li}, q_{Ri}, q_{Mi}, q_{Ci}, q_{Oi}] $

$ g_0 = [0, 0, 1, 0, -1] $

$ g_1 = [0, 0, 1, 0, -1] $

$ g_2 = [0, 0, 1, 0, -1] $

$ g_3 = [2, 0, 0, 0, -1] $

$ g_4 = [0, 0, 1, 0, -1] $

$ g_5 = [1, -1, 0, 0, -1] $

$ g_6 = [1, 0, 0, 3, -1] $

**let x = 2 y = 3**


| Vectors| a | b | c | qLi | qRi | qMi | qCi | qOi |
| --- | --- | --- | --- |--- | --- | --- | --- | --- |
| gate0 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate1 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate2 | 3 | 3 | 9 | 0 | 0 | 1 | 0 | -1 |
| gate3 | 4 | 0 | 8 | 2 | 0 | 0 | 0 | -1 |
| gate4 | 4 | 9 | 36 | 0 | 0 | 1 | 0 | -1 |
| gate5 | 8 | 36 | -28 | 1 | -1 | 0 | 0 | -1 |
| gate6 | -28 | 3 | -25 | 1 | 0 | 0 | 3 | -1 |

In [3]:
import galois
import numpy as np

x = 2
y = 3

out = 2*x**2 - x**2*y**2 + 3
print(out)

a = [2, 2, 3, 4, 4, 8, -28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, -28, -25]

ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, -1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1, -1]

def to_galois_array(a, field):
    # normalize to positive values
    a = [x % field.order for x in a]
    return GF(a)

def to_vanishing_poly(a, field):
    p = galois.Poly([1, -int(a[0]) % field.order], field=field)
    for i in range(1, len(a)):
        p *= galois.Poly([1, -int(a[i]) % field.order], field=field)
    return p

def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if type(v) == list else v
    return galois.lagrange_poly(x, y)

n = len(a)

p = 71
GF = galois.GF(p)
w = GF.primitive_root_of_unity(n)

roots = GF([w**x for x in range(0, n)])

_A = to_poly(roots, a, GF)
_B = to_poly(roots, b, GF)
_C = to_poly(roots, c, GF)
ZH = to_vanishing_poly(roots, GF)
# Z^n - 1 = (Z - 1)(Z - w)(Z - w^2)...(Z - w^(n-1))
ZH = galois.Poly([1, 0, 0, 0, 0, 0, 0, -1], field=GF)

bA = galois.Poly.Random(degree=1, field=GF)
bB = galois.Poly.Random(degree=1, field=GF)
bC = galois.Poly.Random(degree=1, field=GF)

A = _A + bA*ZH
B = _B + bB*ZH
C = _C + bC*ZH

QL = to_poly(roots, ql, GF)
QR = to_poly(roots, qr, GF)
QM = to_poly(roots, qm, GF)
QC = to_poly(roots, qc, GF)
QO = to_poly(roots, qo, GF)

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + qc(x) + c(x)*qo(x)
G = A*QL + B*QR + A*B*QM + QC + C*QO

T = G // ZH

print("--- Vanishing Polynomial ---")
print("ZH = ", ZH)

print("\n\n--- Blinding Polynomials ---")
print(f"bA = {bA}")
print(f"bB = {bB}")
print(f"bC = {bC}")

print("\n\n--- Wire Polynomials ---")
print(f"A = {_A}")
print(f"B = {_B}")
print(f"C = {_C}")

print("\n\n--- Blinded Wire Polynomials ---")
print(f"A' = {A}")
print(f"B' = {B}")
print(f"C' = {C}")

print("\n\n--- Selector Polynomials ---")
print(f"QL = {QL}")
print(f"QR = {QR}")
print(f"QM = {QM}")
print(f"QC = {QC}")
print(f"QO = {QO}")

print("\n\n--- Gate Constraints Polynomial ---")
print(f"G = {G}")
for i in range(0, len(roots)):
    print(f"gate #{i} G({roots[i]}) = {G(roots[i])} --> {'OK' if G(roots[i]) == 0 else 'FAIL'}")

print("\n\n--- Target Polynomial ---")
print(f"T = {T}")

assert G % ZH == 0

-25
--- Vanishing Polynomial ---
ZH =  x^7 + 70


--- Blinding Polynomials ---
bA = 54x + 70
bB = 67x + 6
bC = 26x + 15


--- Wire Polynomials ---
A = 24x^6 + 59x^5 + 22x^4 + 41x^3 + 57x^2 + 33x + 50
B = 19x^5 + 40x^4 + 38x^3 + 43x^2 + 57x + 18
C = 46x^6 + 34x^5 + 14x^4 + 30x^3 + 12x^2 + 19x + 62


--- Blinded Wire Polynomials ---
A' = 54x^8 + 70x^7 + 24x^6 + 59x^5 + 22x^4 + 41x^3 + 57x^2 + 50x + 51
B' = 67x^8 + 6x^7 + 19x^5 + 40x^4 + 38x^3 + 43x^2 + 61x + 12
C' = 26x^8 + 15x^7 + 46x^6 + 34x^5 + 14x^4 + 30x^3 + 12x^2 + 64x + 47


--- Selector Polynomials ---
QL = 20x^5 + 30x^4 + 53x^3 + 21x^2 + 58x + 31
QR = 54x^6 + 36x^5 + 24x^4 + 16x^3 + 58x^2 + 15x + 10
QM = 70x^6 + 19x^5 + 11x^4 + 52x^3 + 2x^2 + 39x + 21
QC = 23x^6 + 51x^5 + 39x^4 + 34x^3 + 26x^2 + 70x + 41
QO = 70


--- Gate Constraints Polynomial ---
G = 3x^22 + 41x^21 + 53x^20 + 12x^19 + 27x^18 + 29x^17 + 38x^16 + 52x^15 + 11x^14 + 34x^13 + 43x^12 + 26x^11 + 53x^10 + 69x^9 + 52x^8 + 27x^7 + 55x^6 + 16x^5 + 18x^4 + 60x^3 + 35x^2 

### Permutation constraint

In [13]:
def print_sigma(sigma, a, b, c, r0, r1, r2):
    r = np.concatenate((r0, r1, r2))
    group_size = len(sigma) // 3
    padding = 6

    print(f"{' w'} | {'value':{padding}} | {'i':{padding}} | {'sigma(i)':{padding}}")

    for i in range(0, group_size):
        print(f"a{i} | {a[i]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    for i in range(group_size, 2 * group_size):
        print(f"b{i - group_size} | {b[i - group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    for i in range(2 * group_size, 3 * group_size):
        print(f"c{i - 2 * group_size} | {c[i - 2 * group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[0], ai[1]: ai[1], ai[2]: ai[2], ai[3]: ci[0], ai[4]: ci[1], ai[5]: ci[3], ai[6]: ci[5],
    bi[0]: bi[0], bi[1]: bi[1], bi[2]: bi[2], bi[3]: bi[3], bi[4]: ci[2], bi[5]: ci[4], bi[6]: bi[6],
    ci[0]: ai[3], ci[1]: ai[4], ci[2]: bi[4], ci[3]: ai[5], ci[4]: bi[5], ci[5]: ai[6], ci[6]: ci[6]
}

k0_roots = roots
k1_roots = roots + GF(n)
k2_roots = roots + GF(2 * n)

print_sigma(sigma, a, b, c, k0_roots, k1_roots, k2_roots)

print("\n\n--- K0 Polynomial ---")
print(f"k0 = {k0_roots}")
print(f"k1 = {k1_roots}")
print(f"k2 = {k2_roots}")

sigma0 = GF([sigma[i] for i in range(0, len(a))])
sigma1 = GF([sigma[i + len(a)] for i in range(0, len(a))])
sigma2 = GF([sigma[i + 2 * len(a)] for i in range(0, len(a))])

S0 = to_poly(k0_roots, sigma0, GF)
S1 = to_poly(k1_roots, sigma1, GF)
S2 = to_poly(k2_roots, sigma2, GF)

SI0 = to_poly(k0_roots, k0_roots, GF)
SI1 = to_poly(k1_roots, k1_roots, GF)
SI2 = to_poly(k2_roots, k2_roots, GF)

beta = GF(2)
gamma = GF(3)

f = (A + SI0 * beta + gamma) * (B + SI1 * beta + gamma) * (C + SI2 * beta + gamma)
g = (A + S0 * beta + gamma) * (B + S1 * beta + gamma) * (C + S2 * beta + gamma)

print("--- f(x) ---")
print(f"f(x) = {f}")

print("\n\n--- g(x) ---")
print(f"g(x) = {g}")

m = f // g

assert m == 1

for i in range(0, len(roots)):
    print(f"m({roots[i]}) = {m(roots[i])} --> {'OK' if z(roots[i]) == 1 else 'FAIL'}")

z = m

print("\n\n--- z(x) ---")
print(f"z(x) = {z}")
print(f"z({w}) = {z(w)}")

 w | value  | i      | sigma(i)
a0 |      2 |      1 |      1
a1 |      2 |     45 |     45
a2 |      3 |     37 |     37
a3 |      4 |     32 |     15
a4 |      4 |     20 |     59
a5 |      8 |     48 |     46
a6 |    -28 |     30 |     62
b0 |      2 |      8 |      8
b1 |      2 |     52 |     52
b2 |      3 |     44 |     44
b3 |      0 |     39 |     39
b4 |      9 |     27 |     51
b5 |     36 |     55 |     34
b6 |      3 |     37 |     37
c0 |      4 |     15 |     32
c1 |      4 |     59 |     20
c2 |      9 |     51 |     27
c3 |      8 |     46 |     48
c4 |     36 |     34 |     55
c5 |    -28 |     62 |     30
c6 |    -25 |     44 |     44


--- K0 Polynomial ---
k0 = [ 1 45 37 32 20 48 30]
k1 = [ 8 52 44 39 27 55 37]
k2 = [15 59 51 46 34 62 44]
--- f(x) ---
f(x) = 64x^24 + 34x^23 + 39x^21 + 11x^20 + 19x^19 + 28x^18 + 52x^16 + 39x^15 + 70x^14 + 14x^13 + 37x^12 + 50x^11 + 12x^10 + 36x^9 + 39x^8 + 23x^7 + 5x^6 + 17x^5 + 27x^4 + 23x^3 + 54x^2 + 2x + 30


--- g(x) ---
g(x) = 

In [5]:
from py_ecc.optimized_bls12_381 import G1, multiply, add

# powers of tau
tau = 45

powers_of_tau = [multiply(G1, tau**i) for i in range(0, G.degree)]

def msm(powers, poly: galois.Poly):
    coeff = poly.coeffs[::-1]
    res = multiply(G1, 0)
    for i in range(0, len(coeff)):
        res = add(res, multiply(powers[i], int(coeff[i])))

    return res

A_commitment = msm(powers_of_tau, A)
B_commitment = msm(powers_of_tau, B)
C_commitment = msm(powers_of_tau, C)
T_commitment = msm(powers_of_tau, T)

print("\n\n--- Commitments ---")
print(f"A_commitment = {A_commitment}")
print(f"B_commitment = {B_commitment}")
print(f"C_commitment = {C_commitment}")
print(f"T_commitment = {T_commitment}")



--- Commitments ---
A_commitment = (1440996239541633621663411071143840779031552511072760568273393438126375140674618049896033985033301919041547535746914, 2676395374917405397458239053962667293580976724032727801251165769329998334187586075429972572291906314663007856842143, 1006794706774446304587937043159097062559042144803409319734372898354694747790216360307314395458182031997957414201513)
B_commitment = (1492147284286448121933453822689291755892366192946460287473602463648628147374677616219307487896060639038787203950314, 3086754946912313891405267948432443605257374645675643929105849189764118848311711412747214306541310563127751532551474, 2014531405745487566126917681276115857417893528833075827929347922439904945545411395870238421867628773817679790822455)
C_commitment = (2170575764428625398746969436559332981893271478845677392644144512290708179624078753398015784178354338035161383414897, 143004867489636231602388887810342693057348394568324641801585596841378296661166589362841484782914971911319115249